### 동적 크롤링
- 동적 크롤링을 위한 selenium을 설치
- pip install selenium 명령어를 사용하여 설치한다.
- crawling을 위한 BeautifulSoup
- 시간 제어를 위한 time
- 출력결과를 저장하기 위한 csv
- selenium을 사용하기 위해서 자신의 크롬 버전과 일치하는 chrome_driver를 설치

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup
import time
import csv

In [ ]:
#삼성병원의 질환백과의 정보를 크롤링함
site = 'http://www.samsunghospital.com/home/healthInfo/content/contentList.do?CONT_CLS_CD=001020001013'

#csv 파일 저장을 위한 f, utf-8을 인코딩으로 사용
f = open('output.csv', 'w',encoding='utf-8')
#파일 객체를 생성
wr = csv.writer(f)

In [ ]:
#현재 PC에 설치한 chromedriver의 위치를 잡아준다.
chrome_driver = 'D:/study/Python/SW/chromedriver.exe'

#chrome으로 driver를 실행
driver = webdriver.Chrome(chrome_driver)

#조건 성립 시간을 위한 wait 객체 설정
wait = WebDriverWait(driver,20)

#driver의 브라우저 크기를 최대화
driver.maximize_window()
#최대화를 하지 않았을 경우 마우스 포인터가 정확히 div를 클릭하지 못하여 크롤링을 못할 수 있음

#웹의 컴포넌트 로딩을 기다리기 위한 implicitly_wait 함수, 로딩이 끝나면 바로 진행된다.
driver.implicitly_wait(3)

#미리 설정해둔 삼성서울병원의 질환백과 사이트로 이동한다.
driver.get(site)

#row의 줄번호를 위한 변수 count
count = 1

#동적 크롤링 할 시에 종료되었는지 판단하기 위한 변수에 마지막 title의 이름을 저장한다.
last_title = driver.find_element_by_css_selector("div.text-list li:last-child").text

while True: 
    #웹의 컴포넌트들을 기다린다.
    driver.implicitly_wait(10)
    time.sleep(1)
    
    #현재 페이지가 있는 title을 가져온다.
    title = driver.find_element_by_css_selector("div.text-list li.active")
    
    #title의 이름을 가져온다.
    title_text = title.text
    
    #동적 크롤링시 다음 칸의 정보를 받기 위해서 사용하는 index객체 i
    i = 1
    
    #마찬가지로 크롤링을 마지막까지 진행했는지 판단하기 위한 변수 마지막 menu의 이름을 가져온다.
    last_menu = driver.find_element_by_css_selector('ul.masonry li:last-child div.card-content-textarea').text
    
    while True:
        
        #웹의 컴포넌트들을 기다린다.
        driver.implicitly_wait(10)
        time.sleep(1)
        
        #menu의 i번째 컴포넌트를 가져온다. 우선 변수가 있기 때문에 따로 문자열을 설정한다.
        menu_selector = 'ul.masonry li:nth-child('+str(i)+') div.card-content-textarea'
        #로딩이 다 끝났는지 확인 후에 (clickable할 수 있다는것은 컴포넌트들이 로딩이 완료됬다는 뜻) 진행한다.
        menu = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, menu_selector)))
        
        #menu의 이름을 가져온다.
        menu_title = menu.text
        
        #클릭하고자 하는 menu가 화면에 없을 시에 선택이 불가능하기 때문에 
        #클릭할 menu의 위치로 스크롤을 이동한다. 
        driver.execute_script("arguments[0].scrollIntoView(true);", menu)
        menu.click()
        
        #웹의 컴포넌트들을 기다린다.
        driver.implicitly_wait(10)
        time.sleep(1)
        
        #로딩이 다 끝났는지 확인후에 질환의 이름, 내용을 가져온다.
        name = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'h1.post-title strong'))).text
        content = driver.find_element_by_css_selector("section.post-detail-body").text
        
        #가져온 질환의 이름, 내용을 csv에 write한다.
        wr.writerow([count, title_text, name, content])
        
        #뒤로가기 키를 눌러서 원래의 title로 돌아간다.
        driver.back()
        
        #총 갯수를 1 늘린다.
        count += 1
        #선택한 메뉴가 마지막 메뉴였을 경우 while문을 빠져나간다.
        if menu_title == last_menu : break;        
        i += 1       
    
    #선택한 title이 마지막 title일 경우 while문을 빠져나간다.
    if title_text == last_title: break
        
    #다음 title을 선택한다.
    next_title = driver.find_element_by_css_selector("div.text-list li.active + li")
    #현재 크롤링을 하며 화면이 아래로 내려가 있기 때문에 제일 위로 스크롤을 올려서 타이틀을 클릭 할 수 있게 한다.
    driver.execute_script('window.scrollTo(0, 0)')
    next_title.click()
    
f.close()
driver.close()

잠복결핵
수족구병
중증열성혈소판감소증
지카바이러스 감염증
성전파성 질환
뇌수막염
대장균 감염
뎅기열
돌발성 발진
만성 C형 간염
말라리아
백일해
브루셀라증
비브리오 패혈증
성홍열
소아마비
수두
신종플루
요충증
유행성 출혈열
인플루엔자
일본뇌염
잠복매독
장염
장티푸스
첨형콘딜로마
칸디다성 구내염
콜레라
파상풍
패혈증
풍진
헤르페스 바이러스 감염
홍역
노로 바이러스 장염
구순포진
결핵
후천성 면역결핍증
간디스토마
감염성 심내막염
골(퇴행성)관절염
류마티스 관절염
근막동통증후군
근이양증
근염
다발성 근육염
요통
척추관 협착증
척추 골절
요추 추간판 탈출증
마르판 증후군
경추관 협착증
경추 추간판 탈출증
후종인대 골화증
각기병
강직성 척추염
갱년기 장애
갑상선 결절
갑상선암
갑상선염
고지혈증
고칼슘혈증
골감소증
골다공증
뇌하수체 기능부전
뇌하수체 기능항진
뇌하수체 종양
당뇨병
부갑상선 기능저하증
부갑상선 기능항진증
부갑상선암
사춘기 조숙
성장호르몬 결핍증
여성형 유방
요붕증
유방암
유방의 섬유선종
유선염
인슐린 비의존성 당뇨병
인슐린 의존성 당뇨병
임신성 당뇨병
자가면역성 갑상선염
쿠싱증후군
갑상선 기능항진증
함몰유두
갑상선 기능저하증
안검경련
사건 수면
주간졸음
주관적 기억장애
일과성 허혈발작
운동실조증
언어 장애
양성 건망증
신경병성 통증 질환
신경 아밀로이드증
샤르코-마리-투스병
손발저림
소뇌실조증
군발두통
경도인지장애
보행장애
근긴장이상
말초신경병증
무도증
기면증
만성 매일 두통
루게릭병
렘수면 행동장애
뇌정맥혈전증
긴장형두통
뇌성마비
인간광우병
강박장애
경계성 인격장애
공황장애
기억상실증
길랑바레증후군
남성 성기능장애
뇌동맥 폐쇄
뇌동맥류
뇌동정맥기형
뇌사
뇌염
뇌전증
뇌졸중
뇌출혈
다발성 경화증
두통
말단비대증
말더듬이
망상장애
모야모야병
반측안면경련
발작
복합부위통증증후군
분리불안증
불면증
불안장애
비정형 파킨슨증
삼차신경병증
섬망
섭식 장애
소아 뇌종양
수막종
수면 무호흡증
수면 장애
신경성 폭식증
실어증
안면 마비
알쯔하이머 치매
알코올 중독
양극성 